# Venn multicalibration

In [6]:

import numpy as np
from data_analysis.data_analysis_utils import *
from VennCalibration.quantile_regression import *
from VennCalibration.venn_multicalibration import *
from VennCalibration.basis_transform import *


 
base_path = "data_analysis/datasets/"
data_name = "meps_21"
random_state = 32
p_train = 0.5
p_cal = 0.3
log_transform_y = False

 


X_train, y_train, X_cal, y_cal, X_test, y_test = prepare_data(data_name, 
                                                                  base_path,
        random_state=random_state,
        p_train=p_train, p_cal=p_cal,
        log_transform_y=log_transform_y)


# compute median predictor as poorly calibrated mean predictor
predictor = quantile_regression(X_train, y_train, alpha = 0.5, test_size = 0.2)
basis_transform = generate_spline_basis_transform(X_cal, n_splines = 5)



output = venn_multicalibration(predictor, X_cal, y_cal, X_test, basis_transform, y_test = y_test)

output
 

,pred_original,pred_calibrated,pred_lower,pred_upper,pred_oracle
0,1.617116,-0.725925,-0.715319,6.488018,-0.700708
1,3.707054,10.378927,10.227502,17.420197,10.227502
2,1.873842,-6.000531,-5.976511,-4.003037,-5.972508
3,7.243047,3.753304,3.713832,8.898633,3.819000
4,2.239594,0.104059,0.103734,1.640053,0.103734
...,...,...,...,...,...
3127,0.000023,-1.098926,-1.090276,2.790084,-1.050922
3128,2.592913,-0.192757,-0.191530,2.945793,-0.178803
3129,1.883390,-4.357733,-4.314327,0.596205,-4.254564
3130,13.608382,8.011550,7.962045,11.008430,7.999120


In [ ]:
import numpy as np

# Seed for reproducibility
np.random.seed(42)

n = 300
# Generate feature values
X = np.sort(np.concatenate([
    np.linspace(0, 10, 150),  # Dense regular points
    np.random.uniform(0, 10, n)  # Additional random points for variability
]))

# Generate response values with some added noise
y = np.sin(X) + np.random.normal(scale=0.5, size=len(X))
 
# Split into training and calibration data
X_train, X_cal, y_train, y_cal = train_test_split(X, y, y_redraw, test_size=0.3, random_state=42)

X_test = X
y_test = np.sin(X) + np.random.normal(scale=0.5, size=len(X))

# Fit a Generalized Additive Model (GAM) to the training data
gam = GAM(s(0, n_splines=10)).fit(X_train.reshape(-1, 1), y_train)


    
# Generate basis transform
basis_transform = generate_spline_basis_transform(X_cal, n_splines=10)
    
# Perform Venn multicalibration
output = venn_multicalibration(predictor, X_cal, y_cal, X_test, basis_transform, y_test=y_test)

In [27]:
import numpy as np
from data_analysis.datasets.datasets import *
from data_analysis.data_analysis_utils import *


from VennCalibration.quantile_regression import quantile_regression
from VennCalibration.basis_transform import generate_spline_basis_transform
from VennCalibration.venn_multicalibration import venn_multicalibration

def report_calibration_errors(data_name, base_path, random_state=32, p_train=0.5, p_cal=0.3, log_transform_y=False):
    """
    Compute and report calibration errors for a given dataset.

    Parameters:
        data_name (str): Name of the dataset.
        base_path (str): Path to the datasets.
        random_state (int): Random seed for reproducibility.
        p_train (float): Proportion of data used for training.
        p_cal (float): Proportion of data used for calibration.
        log_transform_y (bool): Whether to apply log transformation to the response variable.

    Returns:
        np.ndarray: Calibration errors for each prediction type.
    """
    # Prepare data
    X_train, y_train, X_cal, y_cal, X_test, y_test = prepare_data(
        data_name,
        base_path,
        random_state=random_state,
        p_train=p_train,
        p_cal=p_cal,
        log_transform_y=log_transform_y
    )

    from sklearn.preprocessing import MinMaxScaler
    # Initialize the MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Scale to [0,1]
    y_train = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_cal = scaler.transform(y_cal.reshape(-1, 1)).flatten()
    y_test = scaler.transform(y_test.reshape(-1, 1)).flatten()


 

    # Compute median predictor as poorly calibrated mean predictor
    predictor = quantile_regression(X_train, y_train, alpha=0.5, test_size=0.2)
    
    # Generate basis transform
    basis_transform = generate_spline_basis_transform(X_cal, n_splines=10)
    
    # Perform Venn multicalibration
    output = venn_multicalibration(predictor, X_cal, y_cal, X_test, basis_transform, y_test=y_test)

    # Extract predictions from the output
    mu_lower = output['pred_lower'].to_numpy()
    mu_upper = output['pred_upper'].to_numpy()
    mu_cal = output['pred_calibrated'].to_numpy()
    mu = output['pred_original'].to_numpy()
    mu_oracle = output['pred_oracle'].to_numpy()

    # Transform the basis functions
    basis = basis_transform(X_test).T  # Transpose for iteration over basis functions

    # Normalize and center each basis function

    basis_normalized = [
        b  if np.array_equal(np.unique(b), [0, 1])  # Check if b is binary (only 0s and 1s)
        else (b - np.mean(b)) / np.std(b) if np.std(b) > 0  # Normalize if standard deviation > 0
        else 1 + 0 * b  # Handle the case where std(b) == 0
        for b in basis
    ]


    # Calculate the basis-transformed projections
    results = np.array([
        [np.mean(b * (y_test - mu)),
         np.mean(b * (y_test - mu_cal)),
         np.mean(b * (y_test - mu_oracle))]
        for b in basis_normalized
    ])

    # Compute calibration errors for each column of `results`
    cal_errors = np.array([np.sqrt(np.mean(scores**2)) for scores in results.T])
    average_width = np.mean(np.abs(mu_upper - mu_lower))
    average_rel_width = np.mean(np.abs(mu_upper - mu_lower) / (0.01 + np.abs(mu_oracle)))
    IQR = np.abs(np.quantile(y_cal, 0.75) - np.quantile(y_cal, 0.25))
    dims = X_train.shape
    out = np.concatenate([[dims[0], dims[1], average_width, average_rel_width], cal_errors])
    return out

def run_dataset(data_name, base_path, num_reps=100, p_train=0.4, p_cal=0.4, log_transform_y=False):
    seed = 42
    np.random.seed(seed)
    random_states = np.random.randint(0, 1e6, size=num_reps).tolist()
    output_list = [report_calibration_errors(data_name, base_path, random_state=state, p_train=0.4, p_cal=0.4, log_transform_y=False) 
    for state in random_states]
    output = np.hstack([output_list])
    metrics = np.mean(output, axis = 0)
    print(metrics.shape)
    return metrics


# Parameters for the dataset
data_name = "bike"
base_path = "data_analysis/datasets/"


# Report calibration errors
calibration_errors = run_dataset(
    data_name=data_name,
    base_path=base_path,
    num_reps = 3,
    p_train=0.4,
    p_cal=0.4,
    log_transform_y=False
)

print("Calibration Errors:")
print(calibration_errors)




base_path = "data_analysis/datasets/"

results = [run_dataset(data_name, base_path = base_path, log_transform_y = (data_name == "meps_21")) for data_name in ['bike', 'bio', 'star', 'meps_21', 'concrete', 'community']]



(7,)
Calibration Errors:
[4.35400000e+03 1.80000000e+01 9.91097237e-02 2.29201982e+00
 1.74745101e-03 3.68142362e-02 1.38161573e-03]
(7,)
(7,)
(7,)
(7,)
(7,)


/Users/larsvanderlaan/repos/VennCalibration/data_analysis/datasets/datasets.py:335: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  attrib = pd.read_csv(base_path + 'communities_attributes.csv', delim_whitespace = True)


IndexError: boolean index did not match indexed array along axis 1; size of axis is 966 but size of corresponding boolean axis is 798

In [26]:
np.array(results)

array([[4.35400000e+03, 1.80000000e+01, 8.60835181e-03, 1.40815659e-01,
        1.92771848e-03, 1.54358396e-03, 1.51061419e-03],
       [1.82920000e+04, 9.00000000e+00, 1.01994444e-02, 5.87785817e-02,
        7.32763242e-03, 1.00067820e-02, 9.44897369e-03],
       [8.64000000e+02, 3.90000000e+01, 3.26294000e-01, 9.07183352e-01,
        9.83800221e-03, 1.13073099e-02, 7.81692218e-03],
       [6.26200000e+03, 1.39000000e+02, 8.79446188e-03, 4.48441223e-01,
        3.17261017e-03, 1.67217742e-03, 1.61867281e-03],
       [4.12000000e+02, 8.00000000e+00, 1.42989492e-01, 5.76762721e-01,
        7.67783504e-03, 8.09220103e-03, 6.41738216e-03],
       [7.97000000e+02, 1.01000000e+02, 6.65474022e-01, 7.49947255e+00,
        9.89853608e-03, 2.08707233e-02, 5.48287698e-03]])

In [ ]:

# Parameters for the dataset
data_name = "meps_21"
base_path = "data_analysis/datasets/"


# Report calibration errors
calibration_errors = run_dataset(
    data_name=data_name,
    base_path=base_path,
    num_reps = 3,
    p_train=0.5,
    p_cal=0.3,
    log_transform_y=False
)

print("Calibration Errors:")
print(calibration_errors)



# Original design matrix (X) and response vector (y)
X = X_basis#[:, 20:30]  # Exclude intercept column if present
y = y.reshape(-1, 1)  # Ensure y is a column vector
XTX = X.T @ X + lambda_reg * np.eye(X.shape[1])  # Add ridge penalty
XTy = X.T @ y 
XTX_inv = np.linalg.inv(XTX)

# New observation (row) to add to the design matrix
x_new = X[1]  # New row of X (1 x p)
y_new = y[1]  # Corresponding new response value (scalar)
lambda_reg = 1e-3
beta_updated = compute_updated_solution(XTX_inv, XTy, x_new, y_new, lambda_reg, verify=True)


# Verification: beta from the new design matrix
x_new = np.array(x_new).reshape(1, -1)
if isinstance(y_new, (np.ndarray, list)):
    y_new = np.array(y_new).flatten()  # Flatten to 1D
    if y_new.size != 1:
        raise ValueError("y_new must be a scalar or a single-element array.")
    y_new = float(y_new[0])
else:
    y_new = float(y_new)
X_new = np.vstack([X, x_new])  # Updated design matrix
y_new_vec = np.vstack([y, [[y_new]]])  # Updated response vector
XTX_new = X_new.T @ X_new + lambda_reg * np.eye(X_new.shape[1])  # Apply ridge penalty
beta_new = np.linalg.inv(XTX_new) @ (X_new.T @ y_new_vec)
# Print the difference and verify
print("Difference between updated and recomputed beta:", np.mean(np.abs(beta_updated - beta_new)))



In [2]:
from VennCalibration.VennQuantilePredictor import *
from VennCalibration.quantile_regression import *
from VennCalibration.CQR import *
  
predictor_venn = conformal_venn_prediction(X, Y, X, Y, alpha = 0.1)
#predictor_cqr = CQR(X, Y, X, Y, alpha=0.1)
y_median, intervals = predictor_venn(X) 


calculate_coverage_in_bins(bin_ids, intervals, outcome)

(array([3.61733651, 2.27077723, 2.63314652, ..., 3.58470702, 2.25344348,
        3.23384404], shape=(5000,)),
 array([[ 1.60338552,  6.06549911],
        [-2.04960446,  5.94476013],
        [-0.0970772 ,  6.12253074],
        ...,
        [ 1.67944427,  5.73864488],
        [-1.09333973,  6.22431163],
        [ 0.27620818,  6.41545658]], shape=(5000, 2)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from VennCalibration.quantile_regression import *
from VennCalibration.calibrators import *

alpha = 0.1
quantile_predictor = quantile_regression(X, Y, alpha=1 - alpha)
q = quantile_predictor(X)

# Apply isotonic calibration
transform = quantile_calibrator_isotonic(q, Y, alpha=1 - alpha, max_depth=20, num_boost_round=5)

# Sort q for smooth plotting
q_sorted_indices = np.argsort(q)
q_sorted = q[q_sorted_indices]
transform_sorted = transform(q_sorted)

# Create a density plot on the x-axis using KDE
kde = gaussian_kde(q)
q_density = kde(q_sorted)

# Create a secondary axis for the density plot
fig, ax1 = plt.subplots()

# Plot the quantile calibration curve
ax1.plot(q_sorted, transform_sorted, label='Calibrated Quantile', color='blue')
ax1.set_xlabel('Uncalibrated Quantile Predictions')
ax1.set_ylabel('Calibrated Quantile Predictions', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True)

# Create a twin axis sharing the same x-axis for density plot
ax2 = ax1.twinx()
ax2.plot(q_sorted, q_density, label='Density of q', color='orange', alpha=0.7)
ax2.set_ylabel('Density', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

fig.suptitle(f'Quantile Calibration Curve with Density Plot (alpha={alpha})')
fig.legend(loc='upper left')
plt.show()


(array([3.61733651, 2.27077723, 2.63314652, ..., 3.58470702, 2.25344348,
        3.23384404], shape=(5000,)),
 array([[ 1.60338552,  6.06549911],
        [-2.04960446,  5.94476013],
        [-0.0970772 ,  6.12253074],
        ...,
        [ 1.67944427,  5.73864488],
        [-1.09333973,  6.22431163],
        [ 0.27620818,  6.41545658]], shape=(5000, 2)))